In [1]:
from cleaning import data_new, X_train, X_test, y_train, y_test

In [2]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from bayes_opt import BayesianOptimization
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np

# Import other necessary libraries
import warnings
warnings.filterwarnings("ignore")


In [3]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.impute import SimpleImputer
import numpy as np
# Define your training and test data
# ...

# Create an imputer to fill missing values in the input data
imputer = SimpleImputer(strategy='mean')

# Impute missing values in the training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Define the optimization function for GaussianNB
def optimize_nb(priors):
    # Normalize priors so that their sum equals 1
    priors_normalized = np.array([priors] * len(np.unique(y_train)))
    priors_normalized /= priors_normalized.sum()
    
    # Create a GaussianNB model with the given priors
    clf =GaussianNB(priors=priors_normalized)
    
    # Train the model with imputed training data
    clf.fit(X_train_imputed, y_train)
    
    # Make predictions on the imputed test data
    y_pred = clf.predict(X_test_imputed)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Define the parameter bounds for the optimization
pbounds = {'priors': (0, 1)}  # Adjust the range as needed

# Instantiate Bayesian Optimization
nb_bo = BayesianOptimization(
    f=optimize_nb,
    pbounds=pbounds,
    random_state=42,
)

# Run Bayesian optimization
nb_bo.maximize(init_points=5, n_iter=10)

# Print the best hyperparameters and result
print("Best parameters for GaussianNB:", nb_bo.max)


|   iter    |  target   |  priors   |
-------------------------------------
| 1         | 0.4339    | 0.3745    |
| 2         | 0.4339    | 0.9507    |
| 3         | 0.4339    | 0.732     |
| 4         | 0.4339    | 0.5987    |
| 5         | 0.4339    | 0.156     |
| 6         | 0.4339    | 0.5032    |
| 7         | 0.4339    | 0.1075    |
| 8         | 0.4339    | 0.3356    |
| 9         | 0.4339    | 0.9998    |
| 10        | 0.4339    | 1.674e-05 |
| 11        | 0.4339    | 0.7241    |
| 12        | 0.4339    | 0.05731   |
| 13        | 0.4339    | 0.6952    |
| 14        | 0.4339    | 0.7577    |
| 15        | 0.4339    | 0.6526    |
Best parameters for GaussianNB: {'target': 0.43388429752066116, 'params': {'priors': 0.3745401188473625}}


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from bayes_opt import BayesianOptimization
import numpy as np
# Define the preprocessing pipeline
imputer = SimpleImputer(strategy='mean')  # Change strategy if needed
scaler = StandardScaler()  # Optional: standardizes the data

preprocessing_pipeline = Pipeline([
    ('imputer', imputer),
    ('scaler', scaler)
])
# Preprocess the training and test data
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)
def optimize_lr(C):
    clf = LogisticRegression(C=C, solver='liblinear')
    clf.fit(X_train_processed, y_train)
    y_pred = clf.predict(X_test_processed)
    return accuracy_score(y_test, y_pred)
# Define the Bayesian Optimization for LogisticRegression
lr_bo = BayesianOptimization(
    f=optimize_lr,
    pbounds={'C': (0.001, 10)},
    random_state=42,
)

# Run Bayesian Optimization
lr_bo.maximize(init_points=5, n_iter=10)

# Print the best hyperparameters
print("Best parameters for LogisticRegression:", lr_bo.max)


|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.5909    | 3.746     |
| 2         | 0.5868    | 9.507     |
| 3         | 0.5909    | 7.32      |
| 4         | 0.5909    | 5.987     |
| 5         | 0.5909    | 1.561     |
| 6         | 0.5702    | 0.001     |
| 7         | 0.5909    | 6.496     |
| 8         | 0.5909    | 2.584     |
| 9         | 0.5909    | 4.809     |
| 10        | 0.5868    | 8.34      |
| 11        | 0.5909    | 2.03      |
| 12        | 0.5909    | 3.189     |
| 13        | 0.5909    | 5.372     |
| 14        | 0.5909    | 4.277     |
| 15        | 0.5868    | 10.0      |
Best parameters for LogisticRegression: {'target': 0.5909090909090909, 'params': {'C': 3.7460266483547775}}


In [31]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Create an imputer to handle missing values in the data
imputer = SimpleImputer(strategy='mean')  # or use other strategies like 'median' or 'most_frequent'

# Impute missing values in the training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
# Define the best hyperparameter found
best_C = 3.7460266483547775
# Create a logistic regression model using the best hyperparameter
clf = LogisticRegression(C=best_C, solver='liblinear')

# Train the model on your imputed training data
clf.fit(X_train_imputed, y_train)

# Make predictions on the imputed test data
y_pred = clf.predict(X_test_imputed)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print("Best Logistic Regression Model Accuracy:", accuracy)

# Print classification report for further insight
print("Classification Report:")
print(classification_report(y_test, y_pred))


Best Logistic Regression Model Accuracy: 0.5867768595041323
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.72      0.66       136
           1       0.54      0.42      0.47       106

    accuracy                           0.59       242
   macro avg       0.57      0.57      0.57       242
weighted avg       0.58      0.59      0.58       242



In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.impute import SimpleImputer
import numpy as np

# Define your training and test data
# Ensure your data (X_train, X_test, y_train, y_test) is loaded and preprocessed
# ...

# Create an imputer to fill missing values in the input data
imputer = SimpleImputer(strategy='mean')

# Impute missing values in the training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Define the objective function for KNeighborsClassifier
def optimize_knn(n_neighbors, weights):
    # Cast n_neighbors to integer
    n_neighbors = int(n_neighbors)
    
    # Convert weights value to 'uniform' or 'distance' based on threshold
    if weights < 0.5:
        weights_str = 'uniform'
    else:
        weights_str = 'distance'
    
    # Create KNeighborsClassifier model
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights_str)
    
    # Fit the model on the imputed training data
    clf.fit(X_train_imputed, y_train)
    
    # Make predictions on the imputed test data
    y_pred = clf.predict(X_test_imputed)
    
    # Calculate accuracy
    return accuracy_score(y_test, y_pred)

# Define the Bayesian Optimization for KNeighborsClassifier
knn_bo = BayesianOptimization(
    f=optimize_knn,
    pbounds={'n_neighbors': (1, 20), 'weights': (0, 1)},
    random_state=42,
)

# Run Bayesian Optimization
knn_bo.maximize(init_points=5, n_iter=10)

# Convert the best weights value to its string equivalent ("uniform" or "distance")
best_weights = 'uniform' if knn_bo.max['params']['weights'] < 0.5 else 'distance'

# Print the best hyperparameters for KNeighborsClassifier
print("Best parameters for KNeighborsClassifier:", {'n_neighbors': int(knn_bo.max['params']['n_neighbors']), 'weights': best_weights})

# Define the objective function for RandomForestClassifier
def optimize_rf(n_estimators, max_depth):
    # Cast n_estimators and max_depth to integers
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    
    # Create RandomForestClassifier model
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    
    # Fit the model on the imputed training data
    clf.fit(X_train_imputed, y_train)
    
    # Make predictions on the imputed test data
    y_pred = clf.predict(X_test_imputed)
    
    # Calculate accuracy
    return accuracy_score(y_test, y_pred)

# Define the Bayesian Optimization for RandomForestClassifier
rf_bo = BayesianOptimization(
    f=optimize_rf,
    pbounds={'n_estimators': (10, 200), 'max_depth': (3, 30)},
    random_state=42,
)

# Run Bayesian Optimization
rf_bo.maximize(init_points=5, n_iter=10)

# Print the best hyperparameters for RandomForestClassifier
print('Best Parameters:-')
print("LogisticRegression:", lr_bo.max['params'])
print("GaussianNB:", nb_bo.max['params'])
print("RandomForest:", {'n_estimators': int(rf_bo.max['params']['n_estimators']), 'max_depth': int(rf_bo.max['params']['max_depth'])})
print("KNN:", {'n_neighbors': int(knn_bo.max['params']['n_neighbors']), 'weights': best_weights})

|   iter    |  target   | n_neig... |  weights  |
-------------------------------------------------
| 1         | 0.7231    | 8.116     | 0.9507    |
| 2         | 0.7025    | 14.91     | 0.5987    |
| 3         | 0.657     | 3.964     | 0.156     |
| 4         | 0.7438    | 2.104     | 0.8662    |
| 5         | 0.7066    | 12.42     | 0.7081    |
| 6         | 0.7438    | 2.161     | 0.8446    |
| 7         | 0.7149    | 1.475     | 0.0       |
| 8         | 0.6074    | 9.474     | 0.0       |
| 9         | 0.7107    | 6.83      | 1.0       |
| 10        | 0.5785    | 17.97     | 0.04947   |
| 11        | 0.595     | 13.64     | 0.0005843 |
| 12        | 0.6322    | 7.392     | 0.0       |
| 13        | 0.7066    | 2.394     | 0.0       |
| 14        | 0.7149    | 1.0       | 1.0       |
| 15        | 0.7149    | 11.34     | 1.0       |
Best parameters for KNeighborsClassifier: {'n_neighbors': 2, 'weights': 'distance'}
|   iter    |  target   | max_depth | n_esti... |
----------------

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Use the best parameters for KNeighborsClassifier
best_n_neighbors = 2  # From Bayesian Optimization result
best_weights = 'distance'  # From Bayesian Optimization result

# Create an instance of KNeighborsClassifier with the best parameters
knn_clf = KNeighborsClassifier(n_neighbors=best_n_neighbors, weights=best_weights)

# Fit the model on the imputed training data
knn_clf.fit(X_train_imputed, y_train)

# Make predictions on the imputed test data
y_pred_knn = knn_clf.predict(X_test_imputed)

# Calculate accuracy for KNeighborsClassifier
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy of KNeighborsClassifier: {knn_accuracy:.4f}")

# Use the best parameters for RandomForestClassifier
best_n_estimators = 144  # From Bayesian Optimization result
best_max_depth = 19  # From Bayesian Optimization result

# Create an instance of RandomForestClassifier with the best parameters
rf_clf = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth)

# Fit the model on the imputed training data
rf_clf.fit(X_train_imputed, y_train)

# Make predictions on the imputed test data
y_pred_rf = rf_clf.predict(X_test_imputed)

# Calculate accuracy for RandomForestClassifier
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy of RandomForestClassifier: {rf_accuracy:.4f}")


Accuracy of KNeighborsClassifier: 0.7438
Accuracy of RandomForestClassifier: 0.7562
